# Clojure

[Clojure](http://clojure.org/) is a dialect of Lisp that runs in the JVM.
It shares with Lisp the code-as-data philosophy and a powerful macro system.
Clojure is predominantly a functional programming language, and features a rich set of immutable, persistent data structures.
It has strong support for reliable multithreading and concurrency.

In [ ]:
; lazy infinite sequence with recursive definition
(def fib-seq-lazy 
  ((fn rfib [a b] 
     (lazy-seq (cons a (rfib b (+ a b)))))
   0 1))
(take 20 fib-seq-lazy)

In [ ]:
; 'unless' cannot be defined with a function because
; it does not always evaluate both its arguments.
(defmacro unless [pred a b]
  `(if (not ~pred) ~a ~b))
(unless false (println "Will print") (println "Will not print"))

In [ ]:
(defn run [nvecs nitems nthreads niters]
  (let [vec-refs (vec (map (comp ref vec)
                           (partition nitems (range (* nvecs nitems)))))
        swap #(let [v1 (rand-int nvecs)
                    v2 (rand-int nvecs)
                    i1 (rand-int nitems)
                    i2 (rand-int nitems)]
                (dosync
                 (let [temp (nth @(vec-refs v1) i1)]
                   (alter (vec-refs v1) assoc i1 (nth @(vec-refs v2) i2))
                   (alter (vec-refs v2) assoc i2 temp))))
        report #(do.
                 (prn (map deref vec-refs))
                 (println "Distinct:"
                          (count (distinct (apply concat (map deref vec-refs))))))]
    (report)
    (dorun (apply pcalls (repeat nthreads #(dotimes [_ niters] (swap)))))
    (report)))
; When run, we see no values get lost or duplicated in the shuffle.
; There are 36 distinct numbers before and after. 
(run 6 6 6 100000)

## Using the classpath toload a jar

This first cell shows that you get an error if you try to import a class not built-in to BeakerX:

In [1]:
(import com.beaker.BeakerXClasspathTest)
(def classpathTest (new BeakerXClasspathTest))

java.lang.ClassNotFoundException:  com.beaker.BeakerXClasspathTest

The magic `%classpath` allows you to add jars to your kernel.

In [2]:
%classpath add jar demoResources/BeakerXClasspathTest.jar

Added jar: [BeakerXClasspathTest.jar]


In [3]:
(import com.beaker.BeakerXClasspathTest)
(def classpathTest (new BeakerXClasspathTest))

#'beaker_clojure_shell_b5a20c18-9e96-43d8-a1cd-2b079a459780/classpathTest

In [4]:
(BeakerXClasspathTest/staticTest)

static_123

In [5]:
(. classpathTest getObjectTest)

object_123